# This notebook supports the paper _Moment Properties for Certain Classes of Structured Markovian Arrival Processes_
## by Azam Asanjarani, Sophie Hautphenne, and Yoni Nazarathy

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/git/MomentPropertiesCertainMAPs-supp-material`


In [2]:
using LinearAlgebra
using QuadGK #numerical integration

## Example (starting with MMPP (SMMPP))

In [3]:
Q_smmpp = [-6.0 4  2;
          1   -4  3;
          1    1 -2]

λ_vec = [10.0, 20, 30];

In [4]:
p = length(λ_vec)
D_smmpp = Diagonal(λ_vec)
C_smmpp = Q_smmpp - D_smmpp
P_smmpp = inv(-C_smmpp)*D_smmpp
all(sum(P_smmpp, dims = 2) .== 1) #check P_smmpp is stochastic

true

In [5]:
s_smmpp = [sum(Q_smmpp[i,j] for j in setdiff(1:p,i)) for i in 1:p]
println(s_smmpp)
all(s_smmpp .< λ_vec) #check we have a slow MMPP

[6.0, 4.0, 2.0]


true

In [6]:
one_vec = ones(p);
one_vec_2p = ones(2p); #usefull for 2p (MMTCP)

### Stationary Distribution

In [7]:
function stat_dist_Q(Q)
    p = size(Q)[1]
    b = vcat(zeros(p-1), ones(1))
    A = vcat(Q'[1:(p-1),:], ones(p)')
    return (A \ b)'
end
π_vec_smmpp = stat_dist_Q(Q_smmpp)
println(round.(π_vec_smmpp, digits = 5))

[0.14286 0.28571 0.57143]


In [8]:
π_vec_smmpp*Q_smmpp

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 0.0  1.11022e-16  -2.22045e-16

In [9]:
π_vec_smmpp*Q_smmpp #check stationary

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 0.0  1.11022e-16  -2.22045e-16

In [10]:
λ_star_smmpp = π_vec_smmpp*D_smmpp*one_vec

24.285714285714285

In [11]:
α_vec_smmpp = π_vec_smmpp*D_smmpp / λ_star_smmpp
println(round.(α_vec_smmpp, digits = 5))

[0.05882 0.23529 0.70588]


In [12]:
sum(α_vec_smmpp)

1.0

In [13]:
α_vec_smmpp*P_smmpp - α_vec_smmpp #check stationary for discrete time with matrix P

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 0.0  2.77556e-17  0.0

### Deviation Matrices

In [14]:
function fund_matrix(Q)
    p = size(Q)[1]
    π_dist = stat_dist_Q(Q)
    inv(ones(p)*π_dist - Q)
end

function dev_matrix(Q)
    p = size(Q)[1]
    Q⁻ = fund_matrix(Q)
    π_dist = stat_dist_Q(Q)
    Q⁻ - ones(p)*π_dist
end;

In [15]:
Dʰ_smmpp = dev_matrix(Q_smmpp)

3×3 Matrix{Float64}:
  0.122449    0.0163265  -0.138776
 -0.0204082   0.130612   -0.110204
 -0.0204082  -0.0693878   0.0897959

In [16]:
Dʰₜ_smmpp(t) = Dʰ_smmpp*(I-exp(Q_smmpp*t)); # transient deviation matrix

In [17]:
Dʰ_smmpp*one_vec #should be 0 

3-element Vector{Float64}:
 3.608224830031759e-16
 4.85722573273506e-16
 4.3021142204224816e-16

In [18]:
π_vec_smmpp*Dʰ_smmpp #should be 0

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 7.07798e-17  1.57831e-16  2.0631e-16

## MTCP (MMTCP)

In [19]:
function match_mmtcp_to_smmpp(Q, λ)
    p = size(Q)[1]
    s = [sum(Q[i,j] for j in setdiff(1:p,i)) for i in 1:p]
    Λ(i) = [-λ[i]               λ[i]-s[i];
            λ_vec[i]-s_smmpp[i] -λ_vec[i]]
    H(i,j) = Diagonal(fill(Q[i,j], 2))
    Q_mmtcp = zeros(2*p, 2*p)
    for i in 1:p
        for j in 1:p
            #i and j are block indexes the actual indexes are h and k
            h = 2*(i-1)+1
            k = 2*(j-1)+1
            if i == j
                Q_mmtcp[h:h+1,k:k+1] = Λ(i)
            else
                Q_mmtcp[h:h+1,k:k+1] = H(i,j)
            end
        end
    end
    Q_mmtcp
end;

In [20]:
Q_mmtcp = match_mmtcp_to_smmpp(Q_smmpp, λ_vec)

6×6 Matrix{Float64}:
 -10.0    4.0    4.0    0.0    2.0    0.0
   4.0  -10.0    0.0    4.0    0.0    2.0
   1.0    0.0  -20.0   16.0    3.0    0.0
   0.0    1.0   16.0  -20.0    0.0    3.0
   1.0    0.0    1.0    0.0  -30.0   28.0
   0.0    1.0    0.0    1.0   28.0  -30.0

In [21]:
D_mmtcp = Q_mmtcp-Diagonal(Q_mmtcp)

6×6 Matrix{Float64}:
 0.0  4.0   4.0   0.0   2.0   0.0
 4.0  0.0   0.0   4.0   0.0   2.0
 1.0  0.0   0.0  16.0   3.0   0.0
 0.0  1.0  16.0   0.0   0.0   3.0
 1.0  0.0   1.0   0.0   0.0  28.0
 0.0  1.0   0.0   1.0  28.0   0.0

In [22]:
C_mmtcp = Q_mmtcp-D_mmtcp

6×6 Matrix{Float64}:
 -10.0    0.0    0.0    0.0    0.0    0.0
   0.0  -10.0    0.0    0.0    0.0    0.0
   0.0    0.0  -20.0    0.0    0.0    0.0
   0.0    0.0    0.0  -20.0    0.0    0.0
   0.0    0.0    0.0    0.0  -30.0    0.0
   0.0    0.0    0.0    0.0    0.0  -30.0

In [23]:
P_mmtcp = inv(-C_mmtcp)*D_mmtcp

6×6 Matrix{Float64}:
 0.0        0.4        0.4        0.0        0.2       0.0
 0.4        0.0        0.0        0.4        0.0       0.2
 0.05       0.0        0.0        0.8        0.15      0.0
 0.0        0.05       0.8        0.0        0.0       0.15
 0.0333333  0.0        0.0333333  0.0        0.0       0.933333
 0.0        0.0333333  0.0        0.0333333  0.933333  0.0

In [24]:
I - inv(Diagonal(Q_mmtcp))*Q_mmtcp  #Jump chain of MTCP is same as P_mmtcp

6×6 Matrix{Float64}:
 0.0        0.4        0.4        0.0        0.2       0.0
 0.4        0.0        0.0        0.4        0.0       0.2
 0.05       0.0        0.0        0.8        0.15      0.0
 0.0        0.05       0.8        0.0        0.0       0.15
 0.0333333  0.0        0.0333333  0.0        0.0       0.933333
 0.0        0.0333333  0.0        0.0333333  0.933333  0.0

In [25]:
π_vec_mmtcp = stat_dist_Q(Q_mmtcp)
println(round.(π_vec_mmtcp, digits = 3))

[0.071 0.071 0.143 0.143 0.286 0.286]


In [26]:
(π_vec_mmtcp[1:2:end] .+ π_vec_mmtcp[2:2:end])' - π_vec_smmpp # See agrement with stationary of SMMPP

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 5.55112e-17  0.0  1.11022e-16

In [27]:
π_vec_mmtcp*Q_mmtcp #check stationary

1×6 adjoint(::Vector{Float64}) with eltype Float64:
 -1.11022e-16  -1.11022e-16  1.66533e-16  …  -6.66134e-16  3.33067e-16

In [28]:
λ_star_mmtcp = π_vec_mmtcp*D_mmtcp*one_vec_2p

24.285714285714285

In [29]:
λ_star_mmtcp - λ_star_smmpp

0.0

In [30]:
α_vec_mmtcp = π_vec_mmtcp*D_mmtcp/λ_star_mmtcp

1×6 adjoint(::Vector{Float64}) with eltype Float64:
 0.0294118  0.0294118  0.117647  0.117647  0.352941  0.352941

In [31]:
(α_vec_mmtcp[1:2:end] .+ α_vec_mmtcp[2:2:end])' - α_vec_smmpp # See agrement with event-stationary initial phase dist of SMMPP

1×3 adjoint(::Vector{Float64}) with eltype Float64:
 2.77556e-17  0.0  1.11022e-16

In [32]:
α_vec_mmtcp*P_mmtcp - α_vec_mmtcp #check stationary 

1×6 adjoint(::Vector{Float64}) with eltype Float64:
 0.0  0.0  1.38778e-17  1.38778e-17  0.0  0.0

In [33]:
Dʰ_mmtcp = dev_matrix(Q_mmtcp)

6×6 Matrix{Float64}:
  0.0973321    0.0251168    0.0122156   0.0041109  -0.0679331  -0.0708425
  0.0251168    0.0973321    0.0041109   0.0122156  -0.0708425  -0.0679331
 -0.00914769  -0.0112605    0.0793336   0.0512787  -0.0543401  -0.055864
 -0.0112605   -0.00914769   0.0512787   0.0793336  -0.055864   -0.0543401
 -0.00956332  -0.0108448   -0.0343822  -0.0350056   0.0535569   0.0362391
 -0.0108448   -0.00956332  -0.0350056  -0.0343822   0.0362391   0.0535569

In [34]:
Dʰₜ_mmtcp(t) = Dʰ_mmtcp*(I-exp(Q_mmtcp*t)); # transient deviation matrix

# Moment calculation

In [35]:
t = 2.5; # Example time

In [36]:
maxevals = 10^3;

### First moment

$$
M_1(t) = \int_0^t e^{Q u_1} D e^{Q (t - u_1)} \, du_1
$$


In [37]:
#Brute force integration
M1_integrand(u1,t, Q, D) = exp(Q*u1)*D*exp(Q*(t-u1));
M1(t, Q, D) = quadgk((u1)->M1_integrand(u1,t,Q, D), 0, t, maxevals = maxevals)[1];

In [38]:
M1(t, Q_smmpp, D_smmpp)

3×3 Matrix{Float64}:
 8.00875  16.181   33.9123
 8.25365  16.6707  34.892
 8.53936  17.2419  36.0351

##### Time stationary

In [39]:
π_vec_smmpp*M1(t, Q_smmpp, D_smmpp)*one_vec # mean at t with brute force integration

60.7142857142857

In [40]:
λ_star_smmpp*t, λ_star_mmtcp*t # mean at t with result (SMMPP and MTCP obviously match)

(60.71428571428571, 60.71428571428571)

##### Event stationary

In [41]:
α_vec_smmpp*M1(t, Q_smmpp, D_smmpp)*one_vec #event stationary with brute force

61.12724958356358

In [42]:
(α_vec_smmpp*Dʰₜ_smmpp(t) + π_vec_smmpp*t)*D_smmpp*one_vec #event stationary with result

61.12724958356358

In [43]:
(α_vec_mmtcp*Dʰₜ_mmtcp(t) + π_vec_mmtcp*t)*D_mmtcp*one_vec_2p #MMTCP - matches event stationary with result

61.1272495835636

#### Second moments

$$
M_2(t)  = 2 \int_0^t \int_0^{u_2} e^{Q u_1} D e^{Q (u_2 - u_1)} D e^{Q (t - u_2)} \, du_1 \, du_2, \\
$$

$$
\text{Var}_{\boldsymbol{\pi}}(N(t))=\boldsymbol{\pi}M_2(t) \mathbf{1}+\boldsymbol{\pi}M_1(t) \mathbf{1}-(\boldsymbol{\pi}M_1(t) \mathbf{1})^2.
$$

$$
\text{Var}_{\boldsymbol{\pi}}\big(N(t)\big)
=(\lambda^\star+2\, {\boldsymbol{\pi}}D D_Q^{\sharp} D \mathbf{1})\,t- 2 {\boldsymbol{\pi}}D D_Q^{\sharp} D_Q^{\sharp}(t) D \mathbf{1} 
$$

In [44]:
maxevals = 10^3;
I0_2(u1,u2,t,Q,D) = exp(Q*u1)*D*exp(Q*(u2-u1))*D*exp(Q*(t-u2))
I1_2(u2,t,Q,D) = quadgk((u1)->I0_2(u1,u2,t,Q,D), 0, u2; maxevals = maxevals)[1]
I2_2(t,Q,D) = quadgk((u2)->I1_2(u2,t,Q,D), 0, t; maxevals = maxevals)[1]
M2(t,Q,D) = 2*I2_2(t,Q,D)

M2 (generic function with 1 method)

In [45]:
@time M2(t, Q_smmpp, D_smmpp)

  0.941137 seconds (10.58 M allocations: 514.151 MiB, 10.35% gc time, 99.64% compilation time)


3×3 Matrix{Float64}:
 455.88    929.223  2037.74
 483.271   984.562  2153.75
 516.939  1052.54   2295.94

##### Time stationary

In [46]:
function brute_time_stationary_var(t,Q,D)
    p = size(Q)[1]
    π_dist = stat_dist_Q(Q)
    m1 = π_dist*M1(t, Q, D)*ones(p)
    π_dist*M2(t,Q,D)*ones(p) + m1 - m1^2
end;

In [47]:
brute_time_stationary_var(t, Q_smmpp, D_smmpp), brute_time_stationary_var(t, Q_mmtcp, D_mmtcp) # same for SMMPP and MMTCP

(107.01500654015945, 107.01500654017309)

In [48]:
# From formula SMMPP
(λ_star_smmpp +2*π_vec_smmpp*D_smmpp*Dʰ_smmpp*D_smmpp*one_vec)*t - 2π_vec_smmpp*D_smmpp*Dʰ_smmpp*Dʰₜ_smmpp(t)*D_smmpp*one_vec

107.01500654015905

In [49]:
# From formula MTCP
(λ_star_mmtcp +2*π_vec_mmtcp*D_mmtcp*Dʰ_mmtcp*D_mmtcp*one_vec_2p)*t - 2π_vec_mmtcp*D_mmtcp*Dʰ_mmtcp*Dʰₜ_mmtcp(t)*D_mmtcp*one_vec_2p

107.01500654016232

##### Event Stationary

In [50]:
function brute_event_stationary_var(t,Q,D)
    p = size(Q)[1]
    π_dist = stat_dist_Q(Q)
    λ_star = π_dist*D*ones(p)
    α_dist = π_dist*D/λ_star
    m1 = α_dist*M1(t, Q, D)*ones(p)
    α_dist*M2(t,Q,D)*ones(p) + m1 - m1^2
end;

In [51]:
brute_event_stationary_var(t, Q_smmpp, D_smmpp)

106.59731603535238

In [52]:
brute_event_stationary_var(t, Q_mmtcp, D_mmtcp)

105.7714212317951

#### Third moments

In [53]:
#QQQQ - to continue here

In [54]:
Dstar = Dʰ*D*Dʰ - one_vec*πD*Dʰ^2
our_m3_order_3(t) = πD1^3*t^3
our_m3_order_2(t) = 6πD1*πD*Dʰ*D1*t^2 
our_m3_order_1(t) = +6πD*Dstar*D1*t - 6πD*Dʰ*integral_m1(t)*D1
our_m3_order_0(t) = -6πD*Dstar*Dʰₜ(t)*D1;

LoadError: UndefVarError: `Dʰ` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [55]:
our_third_moment(t) = our_m3_order_3(t) + our_m3_order_2(t) + our_m3_order_1(t) + our_m3_order_0(t);

In [56]:
t_range = 1:0.5:10
ours = [our_third_moment(t) for t in t_range]
numerical = [third_moment(t) for t in t_range]
for (i,t) in enumerate(t_range)
    # @show t, ours[i], numerical[i]
    @show t, ours[i] - numerical[i]
end

LoadError: UndefVarError: `our_m3_order_3` not defined in `Main`
Suggestion: check for spelling errors or missing imports.